In [6]:
import json
import xmltodict
import pandas as pd
pd.options.display.max_columns=30000

file_name = '../source/vocabs/source.xml'
save_file = '../source/vocabs/source.json'
languages = [{'0': 'en'}, {'1': 'nl'}, {'2': 'fr'}, {'3': 'de'}, {'4': 'ar'}, {'5': 'it'}, 
             {'6': 'gr'}, {'7': '-'}, {'8': '-'}, {'9': 'sv'}, {'10': 'heb'}, {'11': 'no'}]


with open(file_name, encoding='utf-8') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
    
    records = data_dict['pockets']['pocket']
#with open(save_file, 'w') as outfile:
#    json.dump(data_dict, outfile, indent=2, ensure_ascii=False)

print('Done')

Done


In [7]:
data = data_dict['pockets']['pocket']

matched_dict=list(filter(lambda data:data['@id']=='211',data))
print(matched_dict[0]['record']['term.type']['value'][0]['#text'])

OBJECT


In [8]:

concept_dict = {}
concept_list = []
term_type_list = []

def select_dict(d, lang1, lang2):
    fallback = None
    for item in d:
        if item['@lang'] == lang1:
            return item
        elif item['@lang'] == lang2:
            fallback = item
    return fallback

for pocket in records:
    record = pocket['record']
            
    if 'term' in record.keys():
        concept_dict['selected'] = record['@selected']
        concept_dict['modified'] = record['@modification']
        concept_dict['created'] = record['@created']
        concept_dict['id'] = record['priref']
    

        ## term, check for many
        if 'term' in record.keys():
            if isinstance(record['term']['value'], dict):            
                concept_dict['term'] = record['term']['value']['#text']
                if len(record['term']['value']['@lang']) == 0:
                    concept_dict['term_lang'] = 'nl'
                else:               
                    concept_dict['term_lang'] = record['term']['value']['@lang'][:2]
            elif isinstance(record['term']['value'], list):                
                _term_dict = select_dict(record['term']['value'], 'nl-NL', 'en-US')
                concept_dict['term'] = _term_dict['#text']
                concept_dict['term_lang'] = _term_dict['@lang'][:2]
            concept_dict['term_dict'] = record['term']
            
        ## Term type
        if 'term.type' in record.keys():
            if isinstance(record['term.type'], dict):
                try:
                    _term_type_dict = select_dict(record['term.type']['value'], 'neutral', '0')
                    term_type_list.append(_term_type_dict['#text'])
                    concept_dict['term_type'] = term_type_list
                    _term_type_dict = []
                    term_type_list = []    
                except:
                    # 113374 {'@lang': 'neutral', '#text': 'objectnaam'}
                    print('bulle')
                    concept_dict['term_type'] = record['term.type']['value']['#text']
            elif isinstance(record['term.type'], list):
                for record_type in record['term.type']:
                    _term_type_dict = select_dict(record_type['value'], 'neutral', '0')
                    term_type_list.append(_term_type_dict['#text'])
                    _term_type_dict = []
                concept_dict['term_type'] = term_type_list
                term_type_list = []    
            concept_dict['term_type_dict'] = record['term.type']
            
            
        ## Source
        if 'source' in record.keys():
            if isinstance(record['source'], dict):            
                concept_dict['source_name'] = record['source']['source']
                if 'source.number' in record['source'].keys():
                    concept_dict['source_id'] = record['source']['source.number']
                if 'source.url' in record['source']:
                    concept_dict['source_url'] = record['source']['source.url']
            if isinstance(record['source'], list):            
                concept_dict['source_name'] = record['source'][0]['source']
                if 'source.number' in record['source'][0].keys():
                    concept_dict['source_id'] = record['source'][0]['source.number']
                if 'source.url' in record['source'][0]:
                    concept_dict['source_url'] = record['source'][0]['source.url']
            concept_dict['source_dict'] = record['source']

        
        def get_term_type(record_use_lref):
            record_use_list = []
            term_type_list = []
            
            if isinstance(record_use_lref, str):
                record_use_list.append(record_use_lref)
            elif isinstance(record_use_lref, list):
                record_use_list = record_use_lref                        
            for match_record_id in record_use_list:                                                
                matched_record = list(filter(lambda data:data['@id']== match_record_id, data))
                if 'term.type' in matched_record[0]['record'].keys():
                    matched_type = matched_record[0]['record']['term.type']
                    if isinstance(matched_type, dict):
                        term_type_dict = select_dict(matched_type['value'], 'neutral', '0')                                                                         
                        term_type_list.append(term_type_dict['#text'])
                        term_type_dict = []                                                                            
                        return term_type_list                   
                    if isinstance(matched_type, list):                       
                        for term_type in matched_type:                            
                            term_type_dict = select_dict(term_type['value'], 'neutral', '0')                        
                            term_type_list.append(term_type_dict['#text'])
                            term_type_dict = []                            
                        return term_type_list
                        return str(term_type_list)
        
        if 'use.lref' in record.keys():
            term_type = get_term_type(record['use.lref'])
            #print(record['priref'], record['use.lref'], term_type)
            concept_dict['term_type'] = term_type
            concept_dict['use_id'] = record['use.lref']    
        
        
        if 'use' in record.keys():
            concept_dict['use_dict'] = record['use']                
        if 'used_for.lref' in record.keys():
            concept_dict['used_for_id'] = record['used_for.lref']
        
        if 'broader_term.lref' in record.keys():
            concept_dict['broader_term_id'] = record['broader_term.lref']
        if 'narrower_term.lref' in record.keys():
            concept_dict['narrower_term_id'] = record['narrower_term.lref']
        if 'related_term.lref' in record.keys():
            concept_dict['related_term_id'] = record['related_term.lref']
        if 'equivalent_term.lref' in record.keys():
            concept_dict['equivalent_term_id'] = record['equivalent_term.lref']
        
        concept_list.append(concept_dict)
        concept_dict = {}
        #print(json.dumps(concept_list, indent=2))
        #print('_____________________')

concept_df = pd.DataFrame(concept_list)
concept_df.to_csv('out/concepts.csv', index=False)
concept_df

bulle


,selected,modified,created,id,term,term_lang,term_dict,term_type,use_id,use_dict,term_type_dict,source_name,source_id,source_url,source_dict,used_for_id,broader_term_id,narrower_term_id,related_term_id,equivalent_term_id
0,False,2021-04-08T15:24:31,2012-06-22T09:07:27,3,aalschop,nl,"{'value': {'@invariant': 'false', '@lang': 'nl...","[OBJECT, SUBJECT]",3880,"{'value': {'@invariant': 'false', '@lang': 'nl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,2021-04-29T10:32:51,2012-06-22T09:07:27,4,wever,nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",[ROLE],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'ROLE...",Art & Architecture Thesaurus,300025367,http://vocab.getty.edu/aat/300025367,"{'source': 'Art & Architecture Thesaurus', 'so...",NaN,NaN,NaN,NaN,NaN
2,False,2020-10-27T10:09:40,2012-06-22T09:07:27,5,aambeeld [gereedschap],nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",[OBJECT],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'OBJE...",Art & Architecture Thesaurus,300023119,http://vocab.getty.edu/aat/300023119,"{'source': 'Art & Architecture Thesaurus', 'so...",9907,6158,NaN,NaN,NaN
3,False,2021-05-21T12:06:04,2010-02-24T18:20:49,7,zijde [textiel],nl,"{'value': [{'@invariant': 'false', '@lang': 'e...",[MATER],5959,"{'value': [{'@invariant': 'false', '@lang': 'e...","{'value': [{'@lang': 'neutral', '#text': 'MATE...",Art & Architecture Thesaurus,300243428,http://vocab.getty.edu/aat/300243428,"{'source': 'Art & Architecture Thesaurus', 'so...",NaN,NaN,NaN,NaN,NaN
4,False,2020-12-08T18:19:58,2012-06-22T09:07:27,8,aandenken,nl,"{'value': [{'@invariant': 'false', '@lang': 'e...",[OBJECT],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'OBJE...",Art & Architecture Thesaurus,300235430,http://vocab.getty.edu/aat/300235430,"{'source': 'Art & Architecture Thesaurus', 'so...",NaN,6404,"[7681, 7067, 29970, 1060, 1383, 36301, 29988, ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30799,False,2022-08-26T10:40:18,2022-08-12T16:41:52,113465,stenen graanmolen,nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",[OBJECT],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'OBJE...",Art & Architecture Thesaurus,300200181,http://vocab.getty.edu/aat/300200181,"{'source': 'Art & Architecture Thesaurus', 'so...",NaN,NaN,NaN,NaN,NaN
30800,False,2022-08-31T13:56:46,2022-08-31T13:56:27,113466,5de eeuw [vierde kwart],nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",[PERIOD],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'PERI...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30801,False,2022-09-01T16:03:12,2022-09-01T16:02:33,113467,belt,en,"{'value': {'@invariant': 'false', '@lang': 'en...",[NAMTYPE],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'NAMT...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30802,False,2022-09-14T13:04:25,2022-09-14T13:04:06,113468,poloshirt,nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",[OBJECT],NaN,NaN,"{'value': [{'@lang': 'neutral', '#text': 'OBJE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
##[{'@lang': 'neutral', '#text': 'OBJECT'}, {'@lang': '0', '#text': 'object name'}, {'@lang': '1', '#text': 'objectnaam'}, {'@lang': '2', '#text': "nom d'objet"}, {'@lang': '3', '#text': 'Objektbezeichnung'}, {'@lang': '4', '#text': 'إسم_قطعة.فنية'}, {'@lang': '5', '#text': "nome dell'oggetto"}, {'@lang': '6', '#text': 'ονομασία αντικειμ.'}, {'@lang': '9', '#text': 'föremål namn'}, {'@lang': '10', '#text': 'שם האובייקט'}, {'@lang': '11', '#text': 'genstand navn'}]
## [{'@lang': 'neutral', '#text': 'MATER'}, {'@lang': '0', '#text': 'material'}, {'@lang': '1', '#text': 'materiaal'}, {'@lang': '2', '#text': 'matériel'}, {'@lang': '3', '#text': 'Material'}, {'@lang': '4', '#text': 'المواد'}, {'@lang': '5', '#text': 'materiale'}, {'@lang': '6', '#text': 'υλικό'}, {'@lang': '9', '#text': 'material'}, {'@lang': '10', '#text': 'חומר'}, {'@lang': '11', '#text': 'materiale'}]

In [10]:
key_list = []
for item in data_dict['pockets']['pocket']:    
    item_keys = list(item['record'].keys())    
    key_list.append(item_keys)
keys_df = pd.DataFrame(key_list)
keys_df.drop_duplicates().reset_index().to_csv('out/keys.csv', index=False)